In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.contrib.tensorboard.plugins import projector

In [ ]:
# Load data set
mnist = input_data.read_data_sets('MNIST_data',one_hot = True)
# Number of running 
max_step = 1001
# Number of figures
figure_num = 3000
# File path
dir = "D:/Studying/tensorflow_scripts/"

# Define the session
sess = tf.Session()
# Load the pictures
embedding = tf.Variable(tf.stack(mnist.test.images[:figure_num]),trainable = False, name = 'embedding')

# User-defined function for doing calculation
# tf.summary.scalar is used to record a scalar and assign a name to it.
def variable_summaries(var):
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean',mean) # Mean
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
        tf.summary.scalar('stddev',stddev) # Standard deviation
        tf.summary.scalar('max',tf.reduce_max(var)) # Maximum
        tf.summary.scalar('min',tf.reduce_min(var)) # Minimum
        tf.summary.histogram('histogram',var) # Histogram
# Name scope
with tf.name_scope('Input'):
    # Define two place holders (Each picture is of size 28*28, which is 784, and will be extended as a one-dimensional vector)
    # 'None' means that the value of first dimension can be any value
    x = tf.placeholder(tf.float32,[None, 784],name='x_input')
    # Ten different kinds of numbers in total, ranging from 0 to 9.
    y = tf.placeholder(tf.float32,[None, 10],name='y_input')

# Display the image
with tf.name_scope('input_reshape'):
    # -1 means uncertain number, since at this stage we don't know the exact number of pictures that will be fed into model.
    # Transfrom into 28*28 matrix.
    # Dimension of 1, since we use grey picture not RGB picture.
    image_shaped_input = tf.reshape(x,[-1,28,28,1]) 
    tf.summary.image('input', image_shaped_input, 10)

with tf.name_scope('layer'):
    with tf.name_scope('Weights'):
    # Define a simple network (without hidden layer)
        Weight = tf.Variable(tf.zeros([784,10]),name = 'W')
        variable_summaries(Weight)
    with tf.name_scope('Biase'):
        Biase = tf.Variable(tf.zeros([10]),name = 'B')
        variable_summaries(Biase)
    with tf.name_scope('Wx_plus_b'):
        Wx_plus_b = tf.matmul(x,Weight) + Biase
    with tf.name_scope('softmax'):
        # prediction is of size 1*10 (one_hot)
        prediction = tf.nn.softmax(Wx_plus_b)
        
with tf.name_scope('loss'):
    # Quadratic loss function and gradient descent training method
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = y,logits = tf.matmul(x,Weight) + Biase ))
    tf.summary.scalar('loss',loss)
with tf.name_scope('train'):
    train_step = tf.train.GradientDescentOptimizer(0.2).minimize(loss)
    
# Initialize the variables
sess.run(tf.global_variables_initializer())

with tf.name_scope('accuracy'):
    with tf.name_scope('correct_prediction'):
        # Evaluate the accuracy of the model (argmax return the index of the max value)
        # correction_prediction is a list of type boolean
        correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(prediction,1)) # One-dimensional tensor
    with tf.name_scope('accuracy'):
        # Calculate the accuracy (Transfrom the type into float32 format)
        accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
        tf.summary.scalar('accuracy',accuracy)
                          
# If this metadata file has already exists. then delete it.
if tf.gfile.Exists(dir + 'projector/projector/metadata.tsv'):
    tf.gfile.DeleteRecursively(dir + 'projector/projector/metadata.tsv')
    tf.gfile.MkDir(dir + 'projector/projector')
# Generate metadata file
with open(dir + 'projector/projector/metadata.tsv','w') as f:
    labels = sess.run(tf.argmax(mnist.test.labels[:],1))
    for i in range(figure_num):
        f.write(str(labels[i]) + '\n')
        
# Merge all summaried
merged = tf.summary.merge_all()

projector_writer = tf.summary.FileWriter(dir + 'projector/projector',sess.graph)
saver = tf.train.Saver()
config = projector.ProjectorConfig()
embed = config.embeddings.add()
embed.tensor_name = embedding.name
embed.metadata_path = dir + 'projector/projector/metadata.tsv'
embed.sprite.image_path = dir + 'projector/mnist_10k_sprite.png'
embed.sprite.single_image_dim.extend([28,28])
projector.visualize_embeddings(projector_writer,config)

for i in range(max_step):
    # Every batch uses 100 samples
    batch_samples, batch_labels = mnist.train.next_batch(100)
    run_options = tf.RunOptions(trace_level = tf.RunOptions.FULL_TRACE)
    run_metadata = tf.RunMetadata()
    summary, _ = sess.run([merged, train_step], feed_dict = {x: batch_samples, y: batch_labels},options = run_options, run_metadata = run_metadata)
    projector_writer.add_run_metadata(run_metadata, 'step%03d' % i)
    projector_writer.add_summary(summary,i)
    
    if i%100 == 0:
        acc = sess.run(accuracy, feed_dict = {x:mnist.test.images, y: mnist.test.labels})
        print("Iteration: " + str(i) + ", Testing accuracy: " + str(acc))
        
saver.save(sess, dir + 'projector/projector/a_model.ckpt', global_step = max_step)
projector_writer.close()
sess.close()